In [1]:
import cv2
import numpy as np
import librosa

In [ ]:
# Load YuNet model (download if not already available)
model_path = (
    "models/YUnet/face_detection_yunet_2023mar_int8bq.onnx"  # Update with actual path
)

# Initialize the face detector
detector = cv2.FaceDetectorYN.create(
    model=model_path,
    config="",  # Not required for ONNX models
    input_size=(720, 1280),  # Adjust based on input resolution
    score_threshold=0.9,
    nms_threshold=0.3,
    top_k=5000,
)

# Read an image
image = cv2.imread("../samples/image.png")
print(image.shape)
height, width = image.shape[:2]

# Set the input size based on the image
detector.setInputSize((width, height))

# Detect faces
faces = detector.detect(image)


print(faces)
print(faces[1].shape)

In [ ]:
x, y, w, h = map(int, faces[1][0][:4])
crop_img = image[y : y + h, x : x + w]

crop_img.shape

In [ ]:

cv2.imshow("cropped", crop_img)
cv2.waitKey(0)

In [2]:
def detect_face(
    image: np.array,
    model_path="models/YUnet/face_detection_yunet_2023mar_int8bq.onnx",
) -> np.array:
    
    detector = cv2.FaceDetectorYN.create(
        model=model_path,
        config="",
        input_size=(image.shape[0], image.shape[1]),
        score_threshold=0.9,
        nms_threshold=0.3,
        top_k=5000,
    )
    
    height, width = image.shape[:2]
    
    detector.setInputSize((width, height))
    faces = detector.detect(image)

    x, y, w, h = map(int, faces[1][0][:4])
    cropped_image = image[y:y+h, x:x+w, :]
    return cropped_image

In [3]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return np.array(frames)


video_frames = extract_frames("../samples/subclip.mp4")
print(f"Extracted {len(video_frames)} frames from video.")

Extracted 750 frames from video.


In [ ]:
video_frames[300]

In [ ]:
face = detect_face(video_frames[0])
face.shape

In [ ]:
cv2.imshow("face", face)
cv2.waitKey(0)

In [ ]:
faces[5].shape

In [ ]:
faces = []
cnt = 0
for i, face in enumerate(video_frames):
    cnt += 1
    print(cnt)
    face = detect_face(face)
    faces.append(face)